# Launching ships

Involves: 

* Choosing a target
* Calculating the distance to that target
* TODO: the fuel costs of breaking orbit
* TODO: validate that the faction has enough fuel to run the mission

    


In [1]:
import sys, os
import pandas as pd
import altair as alt
sys.path.append('../..')

import  ssl, asyncio

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio
# this is required for running in a Jupyter Notebook. 

nest_asyncio.apply()

PII_cols = ['username','userguid','owner']

In [4]:
from app.ajaxviews import ships

c = ships.CosmosdbClient()

In [7]:
ship = {"name": "fakeship",
        "objid": "5254363799802"}

shipIsIn = ships.get_ship_isin(c, ship)
shipIsIn

{'id': '04b9cba5-67c8-47e6-afcc-5b08d8c2a5e7',
 'label': 'isIn',
 'type': 'edge',
 'inVLabel': 'building',
 'outVLabel': 'ship',
 'inV': '4254954610763',
 'outV': '5254363799802',
 'properties': {'userguid': '8d5b667f-b225-4641-b499-73b77558ff86'}}

In [9]:
location_query = (
    f"""g.V().has('objid','{shipIsIn['inV']}')
            .in('owns')
            .out('inhabits')
            .valueMap()
    """
)
c.run_query(location_query)
origin_location = c.clean_nodes(c.res)[0]
origin_location

{'name': 'Karnia',
 'class': 'terrestrial',
 'objid': '3366473281713',
 'pop_cap': 100,
 'atmosphere': '[{Argon: 0.108}, {Carbon Dioxide: 0.357}, {Helium: 0.066}, {Hydrogen: 0.172}, {Methane: 0.002}, {Nitrogen: 0.15}, {Oxygen: 0.065}, {Sodium: 0.079}]',
 'radius': 0.919,
 'mass': 0.392,
 'orbitsDistance': 0.631,
 'orbitsId': '7510190479449',
 'orbitsName': 'Hesroycia',
 'isSupportsLife': 'true',
 'isPopulated': 'true',
 'isHomeworld': 'true',
 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
 'objtype': 'planet',
 'id': '3366473281713'}

In [10]:
target_options = [{'name': 'Karnia',
  'class': 'terrestrial',
  'objid': '3366473281713',
  'pop_cap': 100,
  'atmosphere': '[{Argon: 0.108}, {Carbon Dioxide: 0.357}, {Helium: 0.066}, {Hydrogen: 0.172}, {Methane: 0.002}, {Nitrogen: 0.15}, {Oxygen: 0.065}, {Sodium: 0.079}]',
  'radius': 0.919,
  'mass': 0.392,
  'orbitsDistance': 0.631,
  'orbitsId': '7510190479449',
  'orbitsName': 'Hesroycia',
  'isSupportsLife': 'true',
  'isPopulated': 'true',
  'isHomeworld': 'true',
  'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
  'objtype': 'planet',
  'id': '3366473281713'},
 {'name': 'Lakarau',
  'class': 'rocky',
  'objid': '9826600357817',
  'pop_cap': 100,
  'orbitsId': '3366473281713',
  'orbitsName': 'Karnia',
  'orbitsDistance': 0.9191,
  'mass': 0.0004,
  'radius': 0.1154,
  'isSupportsLife': 'false',
  'isPopulated': 'false',
  'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
  'objtype': 'moon',
  'id': '9826600357817'}]

In [39]:
def get_distance(target):
    if target["objtype"] == "planet":
        total_distance = abs(target['orbitsDistance'] - origin_location['orbitsDistance'])
    if target["objtype"] == "moon":
        c.run_query(f"g.V().has('objid','{target['orbitsId']}').valueMap()")
        orbit_planet = c.clean_nodes(c.res)[0]
        total_distance = abs(orbit_planet['orbitsDistance'] - origin_location['orbitsDistance']) + target['orbitsDistance']
        # TODO: Arbitrarily dividing the distance by 100 to get a more reasonable number. I should fix this in genesis so that moons orbitdistance is calculated by AU.
        total_distance = total_distance/100
    return total_distance

get_distance(target_options[1])

0.009191

In [40]:
get_distance(target_options[0])

0.0